In [1]:
import importlib 
import modules
from esm.models.esm3 import ESM3
from esm.sdk.api import ESMProtein, GenerationConfig
import pytorch_lightning as L
import json 
import utils
import torchtune
import torch
import VirusDataset
import os
importlib.reload(modules) 
importlib.reload(utils)

/home/tyfei/anaconda3/envs/esm3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<module 'utils' from '/home/tyfei/evoModel/utils.py'>

In [ ]:
import tensorboard

In [6]:
a = torch.randn((4, 5))
b = torch.tensor([1,3,2,0]) 
c = torch.nn.CrossEntropyLoss()
c(a, b).shape
# torch.sum(a).shape

torch.Size([])

In [10]:
with open(os.path.join("/home/tyfei/evoModel/checkpoints/trainAutoencoder", "config.json"), "r") as f:
    configs = json.load(f)
# json_formatted_str = json.dumps(configs, indent=2)

In [11]:
model = utils.loadesm3(configs)

In [12]:
ae = modules.AutoEncoder(model)

In [13]:
ds = utils.loadDatasetesm3(configs)

EOFError: Ran out of input

In [ ]:
checkpoint_callback = ModelCheckpoint(
        monitor="validate_acc",  # Replace with your validation metric
        mode="max",  # 'min' if the metric should be minimized (e.g., loss), 'max' for maximization (e.g., accuracy)
        save_top_k=k,  # Save top k checkpoints based on the monitored metric
        save_last=True,  # Save the last checkpoint at the end of training
        dirpath=args.path,  # Directory where the checkpoints will be saved
        filename="{epoch}-{validate_acc:.2f}",  # Checkpoint file naming pattern
    )


trainer = L.Trainer(
        logger=logger,
        accelerator="gpu",
        # profiler=profiler,
        devices=[5],
        max_epochs=configs["train"]["epoch"],
        log_every_n_steps=1,
        accumulate_grad_batches=configs["train"]["accumulate_grad_batches"],
        callbacks=cbs,
    )

In [86]:
ae.training_step((x, target, mask), 1)

tensor(97.9728, grad_fn=<DivBackward0>)

In [5]:
f = open("/data/tyfei/datasets/Spike/standard.txt")
standard = f.readline().strip() 
f.close()
standard

'MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQDVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSPRRARSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKDFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTASALGKLQDVVNQNAQALNTLVKQLSSNFGAISSVLNDILSRLDKVEAEVQIDRLITG

In [6]:
protein = ESMProtein(sequence=standard)

In [15]:
model.transformer.blocks

ModuleList(
  (0): UnifiedTransformerBlock(
    (attn): MultiHeadAttention(
      (layernorm_qkv): Sequential(
        (0): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (1): Linear(in_features=1536, out_features=4608, bias=False)
      )
      (out_proj): Linear(in_features=1536, out_features=1536, bias=False)
      (q_ln): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
      (k_ln): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
      (rotary): RotaryEmbedding()
    )
    (geom_attn): GeometricReasoningOriginalImpl(
      (s_norm): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
      (proj): Linear(in_features=1536, out_features=3840, bias=False)
      (out_proj): Linear(in_features=768, out_features=1536, bias=False)
    )
    (ffn): Sequential(
      (0): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
      (1): Linear(in_features=1536, out_features=8192, bias=False)
      (2): SwiGLU()
      (3): Linear(in_features=4096, out_feature

In [80]:
model = model.cpu()
res = model.encode(protein)
# res.sequence[None, :]
# res = model.forward(sequence_tokens=res.sequence[None, :])

In [43]:
emb = res.embeddings#.shape
emb

tensor([[[ 158.0931,  113.4213,  -16.8635,  ...,   13.8235,   11.9271,
             8.1903],
         [ 107.5984,   49.2519,  -42.7444,  ..., -217.9072,    7.2081,
            26.4612],
         [ 206.0871,   13.8621,  -37.0269,  ..., -158.8261,  -27.1618,
            10.8131],
         ...,
         [ 118.6670,   15.5755, -115.0011,  ...,   47.9468,  253.2842,
           170.3626],
         [ 223.6516,  182.5809,    6.9576,  ...,   17.4236,  234.5673,
           127.9122],
         [  79.4623,  153.3872,  -67.3726,  ...,   20.7665,   99.0743,
           165.0842]]], grad_fn=<AddBackward0>)

In [8]:
protein = model.generate(
                    protein,
                    GenerationConfig(track="secondary_structure", num_steps=16),
)
protein = model.generate(
    protein, GenerationConfig(track="sasa", num_steps=16)
)
protein = model.generate(
    protein, GenerationConfig(track="structure", num_steps=16)
)

  0%|          | 0/16 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

InternalTorchDynamoError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


from user code:
   File "/home/tyfei/anaconda3/envs/esm3/lib/python3.10/site-packages/esm/utils/structure/affine3d.py", line 325, in _graham_schmidt
    with fp32_autocast_context(x_axis.device.type):

Set TORCH_LOGS="+dynamo" and TORCHDYNAMO_VERBOSE=1 for more information


You can suppress this exception and fall back to eager by setting:
    import torch._dynamo
    torch._dynamo.config.suppress_errors = True


In [14]:
res = model.encode(protein)
data = {}
            
data["seq_t"] = res.sequence.cpu().numpy()
data["structure_t"] = res.structure.cpu().numpy()
data["second_t"] = res.secondary_structure.cpu().numpy()
data["sasa_t"] = res.sasa.cpu().numpy()
# data["coordinates"] = res.coordinates.cpu().numpy()

In [20]:
data["seq_t"][None, :].repeat(20, 1).shape

torch.Size([20, 1275])

In [23]:
representations = model(
            sequence_tokens=data["seq_t"][None, :].repeat(2, 1),
            structure_tokens=data["structure_t"][None, :].repeat(2, 1),
            ss8_tokens=data["second_t"][None, :].repeat(2, 1),
            sasa_tokens=data["sasa_t"][None, :].repeat(2, 1),
        )
representations

OutOfMemoryError: CUDA out of memory. Tried to allocate 9.30 GiB. GPU  has a total capacity of 23.55 GiB of which 6.87 GiB is free. Including non-PyTorch memory, this process has 16.65 GiB memory in use. Of the allocated memory 11.61 GiB is allocated by PyTorch, and 4.59 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [17]:
model()

{'seq_t': array([ 0, 20, 18, ..., 19, 11,  2]),
 'structure_t': array([4098,  247, 3611, ...,  808,  124, 4097]),
 'second_t': array([ 0, 10, 10, ..., 10, 10,  0]),
 'sasa_t': array([ 0, 18, 18, ..., 18, 18,  0]),
 'coordinates': array([[[         inf,          inf,          inf],
         [         inf,          inf,          inf],
         [         inf,          inf,          inf],
         ...,
         [         inf,          inf,          inf],
         [         inf,          inf,          inf],
         [         inf,          inf,          inf]],
 
        [[ 61.731552  ,  -6.571238  ,  16.325504  ],
         [ 62.342125  ,  -7.4330754 ,  15.318746  ],
         [ 61.35109   ,  -8.477322  ,  14.815431  ],
         ...,
         [         inf,          inf,          inf],
         [         inf,          inf,          inf],
         [         inf,          inf,          inf]],
 
        [[ 61.160416  ,  -8.275962  ,  15.921618  ],
         [ 61.024757  ,  -9.643171  ,  15.430236